# <center>Классификация рукописных цифр

In [1]:
import numpy as np

import keras
from keras.datasets import mnist

Using TensorFlow backend.


Загрузим обучающую и тестовую выборки.

In [42]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Для распознавания рукописных цифр будем использовать полносвязную нейронную сеть, где каждый нейрон входного слоя характеризует один пиксель исходного изображения. При этом сами изображения представляют собой матрицу чисел, поэтому для дальнейшей работы необходимо выполнить преобразования над данными.

С помощью функции `reshape` преобразуем каждую матрицу в вектор длины 28`*`28 (размер исходных изображений).

In [41]:
#  черновик

# y_train.shape
# x_test.shape[0]
# y_train
# x_train.reshape(60000,28*28).shape

(60000,)

In [43]:
x_train, x_test =  x_train.reshape(x_train.shape[0],28*28), x_test.reshape(x_test.shape[0],28*28)

Поскольку мы будем использовать градиентные методы обучения, необходимо также нормализовать данные.

In [44]:
np.max(x_train) #максимальная интенсивность цвета

255

In [45]:
x_train = x_train / 255 
x_test = x_test / 255

В данный момент значение целевой переменной каждого изображения представляет собой цифру от 0 до 9. Для решения задачи многоклассовой классификации можно использовать функцию softmax. Её выход при этом - вектор вероятностей принадлежности к каждому из классов. Таким образом, возникает необходимость преобразовать `y_train` и `y_test`.

Для этих целей в `keras` доступна функция `keras.utils.to_categorical`, принимающая на вход исходный вектор значений и количество классов. Применим данную функцию к  `y_train` и  `y_test`.

In [46]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [7]:
y_train[0].shape # 10 классов, всё ок

(10,)

##  Полносвязная нейронная сеть

Теперь можно приступать к обучению сети. Будем использовать следующую архитектуру:
- на вход подается 784 значения;
- далее идет полносвязный слой с 256 нейронами и функцией активации `relu`;
- для повышения обобщающей способности модели добавим `Dropout(0.2)`, где `0.2` - вероятность отключения каждого из нейронов слоя;
- далее еще один полносвязный слой с 256 нейронами и функцией активации `relu`;
- последний слой - выход сети - полносвязный слой с числом нейронов, равным числу классов и функцией активации `softmax`.

In [8]:
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout

In [9]:
28*28

784

In [10]:
model = Sequential() 

model.add(Dense(256, input_shape=(28*28,), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

Обучим полученную модель на обучающей выборке. Для обучения:
- функция потерь: https://keras.io/losses/; 
- optimizer - RMSProp: https://keras.io/optimizers/; 
- размер мини-батча - 128;
- число эпох - 10;
- на каждом шаге выводите значение `accuracy` как на обучающей, так и на тестовой выборках.

In [11]:
model.compile(optimizer='RMSProp', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=128)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 16s 266us/step - loss: 0.2810 - accuracy: 0.9154 - val_loss: 0.1229 - val_accuracy: 0.9641
Epoch 2/10
60000/60000 [==============================] - 11s 186us/step - loss: 0.1171 - accuracy: 0.9639 - val_loss: 0.0889 - val_accuracy: 0.9743
Epoch 3/10
60000/60000 [==============================] - 12s 202us/step - loss: 0.0864 - accuracy: 0.9730 - val_loss: 0.0903 - val_accuracy: 0.9714
Epoch 4/10
60000/60000 [==============================] - 12s 208us/step - loss: 0.0691 - accuracy: 0.9784 - val_loss: 0.0829 - val_accuracy: 0.9751
Epoch 5/10
60000/60000 [==============================] - 12s 206us/step - loss: 0.0589 - accuracy: 0.9814 - val_loss: 0.0602 - val_accuracy: 0.9822
Epoch 6/10
60000/60000 [==============================] - 12s 207us/step - loss: 0.0488 - accuracy: 0.9841 - val_loss: 0.0687 - val_accuracy: 0.9805
Epoch 7/10
60000/60000 [==============================] 

In [13]:
score = model.evaluate(x_test, y_test, batch_size=128)
score

10000/10000 [==============================] - 1s 75us/step


[0.07092731841329951, 0.9830999970436096]

In [14]:
model.layers

Попробуем улучшить полученный результат, меняя архитектуру сети.

In [51]:
model2 = Sequential() 
model2.add(Dense(512, input_shape=(28*28,), activation='relu'))
model2.add(Dropout(0.25))
model2.add(Dense(256, activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(128, activation='relu'))
model2.add(Dense(64, activation='relu'))
#model2.add(Dense(32, activation='relu'))
model2.add(Dense(10, activation='softmax'))

In [52]:
model2.compile(optimizer='RMSProp', loss='categorical_crossentropy', metrics=['accuracy'])

In [53]:
model2.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=15, batch_size=128)

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 27s 453us/step - loss: 0.3042 - accuracy: 0.9053 - val_loss: 0.1212 - val_accuracy: 0.9634
Epoch 2/15
60000/60000 [==============================] - 24s 392us/step - loss: 0.1278 - accuracy: 0.9621 - val_loss: 0.0901 - val_accuracy: 0.9730
Epoch 3/15
60000/60000 [==============================] - 25s 410us/step - loss: 0.0951 - accuracy: 0.9712 - val_loss: 0.0743 - val_accuracy: 0.9790
Epoch 4/15
60000/60000 [==============================] - 25s 418us/step - loss: 0.0780 - accuracy: 0.9773 - val_loss: 0.0764 - val_accuracy: 0.9784
Epoch 5/15
60000/60000 [==============================] - 28s 461us/step - loss: 0.0674 - accuracy: 0.9801 - val_loss: 0.0743 - val_accuracy: 0.9793: 0
Epoch 6/15
60000/60000 [==============================] - 30s 500us/step - loss: 0.0602 - accuracy: 0.9822 - val_loss: 0.0805 - val_accuracy: 0.9781
Epoch 7/15
60000/60000 [=============================

In [54]:
score2 = model2.evaluate(x_test, y_test, batch_size=128)
score2

10000/10000 [==============================] - 2s 151us/step


[0.06898705225066151, 0.9854000210762024]

##  Свёрточная нейронная сеть

**Вариант 3**

In [18]:
from keras.layers import Conv2D, Flatten, MaxPool2D

In [26]:
model4 = Sequential() 

model4.add(Conv2D(10, kernel_size=5, activation='relu', input_shape=(28,28,1)))
#model4.add(Conv2D(10, kernel_size=5, activation='relu'))
model4.add(MaxPool2D(pool_size=(2,2)))
model4.add(Conv2D(20, kernel_size=5, activation='relu'))
model4.add(MaxPool2D(pool_size=(2,2)))
model4.add(Dropout(0.25))
model4.add(Flatten())
model4.add(Dense(100, activation='relu'))
model4.add(Dense(10, activation='softmax'))

In [27]:
model4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [31]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train.reshape(x_train.shape[0], 28, 28, 1), x_test.reshape(x_test.shape[0], 28, 28, 1)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [32]:
model4.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 75s 1ms/step - loss: 0.5429 - accuracy: 0.8841 - val_loss: 0.0958 - val_accuracy: 0.9714
Epoch 2/5
60000/60000 [==============================] - 76s 1ms/step - loss: 0.1285 - accuracy: 0.9611 - val_loss: 0.0660 - val_accuracy: 0.9780
Epoch 3/5
60000/60000 [==============================] - 77s 1ms/step - loss: 0.0992 - accuracy: 0.9702 - val_loss: 0.0487 - val_accuracy: 0.9851
Epoch 4/5
60000/60000 [==============================] - 88s 1ms/step - loss: 0.0845 - accuracy: 0.9747 - val_loss: 0.0450 - val_accuracy: 0.9850
Epoch 5/5
60000/60000 [==============================] - 91s 2ms/step - loss: 0.0748 - accuracy: 0.9775 - val_loss: 0.0425 - val_accuracy: 0.9853


In [33]:
score3 = model4.evaluate(x_test, y_test, batch_size=128)
score3

10000/10000 [==============================] - 5s 470us/step


[0.04247035065311938, 0.9853000044822693]

### Вывод
1. Были рассмотрены полносвязные и свёрточные неейронные сети.
2. В полносвязных нейронных сетях увеличение числа слоёв и нейронов позволило добиться высокой точности классификации модели (accuracy = 0.9854).
3. Свёрточная нейронная сеть позволила добиться точности классификации (accuracy = 0.9853)